### **Curating face_forehead1a.h5ad**

Article:  Multi-scale spatial mapping of cell populations across anatomical sites in healthy human skin and basal cell carcinoma

DOI: https://doi.org/10.1101/2023.08.08.551504

Data Source : https://spatial-skin-atlas.cellgeni.sanger.ac.uk

##### **Mount farm**

mount-farm

##### **Packages required for curation**

In [1]:
#Import all packages required for curation

In [2]:
import numpy as np
import pandas as pd
import scanpy as sc
import scipy
from tqdm import tqdm
from scipy import sparse
from scipy.sparse import csr_matrix
import anndata as ad
import os
import subprocess
import math

### **Curation Schema**

##### **X (Matrix Layers)**

##### **AnnData object**

In [3]:
# Load the AnnData object

In [4]:
adata = sc.read_h5ad('/lustre/scratch127/cellgen/cellgeni/cxgportal_sets/spatial-skin/Data/WSSKNKCLsp10446619.h5ad')

In [5]:
# View the AnnData object

In [6]:
adata

AnnData object with n_obs × n_vars = 1163 × 17030
    obs: 'in_tissue', 'array_row', 'array_col', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb', 'pct_counts_rb', 'barcode', 'c2l_Th', 'c2l_NK', 'c2l_APOD+ fibroblasts', 'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2', 'c2l_DC1', 'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes', 'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc', 'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes', 'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts', 'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC', 'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes', 'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1'
    uns: 'leiden', 'log1p', 'neighbors', 'pca', 'spatial', 'umap'
    obsm: 'X_spatial'

##### **X- expression matrix**

In [7]:
# View the expression matrix of the anndata object

In [8]:
adata.X

array([[-0.05406324, -0.05398596, -0.19418664, ..., -0.07032977,
        -0.37806773, -0.09116589],
       [-0.05406324, -0.05398596, -0.19418664, ..., -0.07032977,
        -0.37806773, -0.09116589],
       [-0.05406324, -0.05398596, -0.19418664, ..., -0.07032977,
        -0.37806773, -0.09116589],
       ...,
       [-0.05406324, -0.05398596, -0.19418664, ..., -0.07032977,
        -0.37806773, -0.09116589],
       [-0.05406324, -0.05398596, -0.19418664, ..., -0.07032977,
        -0.37806773, -0.09116589],
       [-0.05406324, -0.05398596, -0.19418664, ..., -0.07032977,
        -0.37806773, -0.09116589]], dtype=float32)

In [9]:
#data type of adata.X

In [10]:
type(adata.X)

numpy.ndarray

In [11]:
#convert to csr_matrix

In [12]:
adata.X = csr_matrix(adata.X)

In [13]:
adata.X

<1163x17030 sparse matrix of type '<class 'numpy.float32'>'
	with 19805890 stored elements in Compressed Sparse Row format>

In [14]:
# Print the matrix to check whether they are normalized counts or raw counts. if the matrix has floating numbers,they are normalized counts.if they are integers, they are raw counts.

In [15]:
print(adata.X)

  (0, 0)	-0.05406324
  (0, 1)	-0.053985957
  (0, 2)	-0.19418664
  (0, 3)	-0.25168204
  (0, 4)	-0.05526896
  (0, 5)	-0.1141215
  (0, 6)	-0.04873102
  (0, 7)	-0.123650126
  (0, 8)	1.7452631
  (0, 9)	-0.17534477
  (0, 10)	-0.28121853
  (0, 11)	-0.18437728
  (0, 12)	2.161574
  (0, 13)	-0.37001887
  (0, 14)	-0.35205886
  (0, 15)	-0.14302252
  (0, 16)	-0.33613813
  (0, 17)	-0.24024545
  (0, 18)	-0.07346289
  (0, 19)	-0.31037733
  (0, 20)	-0.24228649
  (0, 21)	-0.70899385
  (0, 22)	-0.27950823
  (0, 23)	-0.41841432
  (0, 24)	-0.060605567
  :	:
  (1162, 17005)	-0.17979954
  (1162, 17006)	-0.10293317
  (1162, 17007)	-0.11377173
  (1162, 17008)	4.3091965
  (1162, 17009)	-0.105500765
  (1162, 17010)	-0.23616515
  (1162, 17011)	-0.3251745
  (1162, 17012)	-0.056089826
  (1162, 17013)	0.29123262
  (1162, 17014)	0.39044118
  (1162, 17015)	-0.072233886
  (1162, 17016)	0.27157006
  (1162, 17017)	-0.38427204
  (1162, 17018)	0.11271776
  (1162, 17019)	0.25196004
  (1162, 17020)	0.1911724
  (1162, 17021)	

##### **Raw counts matrix**

In [16]:
# If X has normalized counts, check for the raw counts matrix.

In [17]:
#Here the raw counts are provided in a separate object, load the raw counts matrix

In [18]:
araw= sc.read_10x_h5('/lustre/scratch127/cellgen/cellgeni/cxgportal_sets/spatial-skin/Raw/all/WSSKNKCLsp10446619/filtered_feature_bc_matrix.h5')

/nfs/users/nfs_s/sa32/anaconda/envs/shibla_env/lib/python3.11/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [19]:
# view raw object

In [20]:
araw

AnnData object with n_obs × n_vars = 1163 × 36601
    var: 'gene_ids', 'feature_types', 'genome'

In [21]:
# view raw matrix

In [22]:
araw.X

<1163x36601 sparse matrix of type '<class 'numpy.float32'>'
	with 2357138 stored elements in Compressed Sparse Row format>

In [23]:
print(araw.X)

  (0, 24)	1.0
  (0, 29)	3.0
  (0, 48)	1.0
  (0, 51)	2.0
  (0, 55)	2.0
  (0, 57)	1.0
  (0, 59)	2.0
  (0, 62)	2.0
  (0, 101)	2.0
  (0, 104)	2.0
  (0, 143)	2.0
  (0, 170)	2.0
  (0, 176)	2.0
  (0, 183)	1.0
  (0, 208)	1.0
  (0, 219)	4.0
  (0, 221)	1.0
  (0, 237)	1.0
  (0, 241)	1.0
  (0, 250)	2.0
  (0, 264)	1.0
  (0, 297)	1.0
  (0, 300)	1.0
  (0, 350)	1.0
  (0, 367)	1.0
  :	:
  (1162, 36101)	1.0
  (1162, 36112)	1.0
  (1162, 36325)	1.0
  (1162, 36329)	1.0
  (1162, 36337)	1.0
  (1162, 36377)	1.0
  (1162, 36378)	3.0
  (1162, 36389)	1.0
  (1162, 36398)	2.0
  (1162, 36400)	1.0
  (1162, 36401)	23.0
  (1162, 36413)	2.0
  (1162, 36450)	2.0
  (1162, 36513)	1.0
  (1162, 36559)	17.0
  (1162, 36560)	11.0
  (1162, 36561)	10.0
  (1162, 36562)	22.0
  (1162, 36564)	13.0
  (1162, 36565)	25.0
  (1162, 36566)	12.0
  (1162, 36567)	1.0
  (1162, 36568)	19.0
  (1162, 36569)	4.0
  (1162, 36571)	11.0


In [24]:
# since the raw object is combined one, extract the raw counts for this dataset 

##### **Variables(var)**

In [25]:
#View the var of anndata and raw object

In [26]:
adata.var

""
AL627309.1
AL627309.5
LINC01409
LINC01128
LINC00115
...
MT-CYB
AC145212.1
MAFIP
AC007325.4


In [27]:
araw.var

,gene_ids,feature_types,genome
MIR1302-2HG,ENSG00000243485,Gene Expression,GRCh38
FAM138A,ENSG00000237613,Gene Expression,GRCh38
OR4F5,ENSG00000186092,Gene Expression,GRCh38
AL627309.1,ENSG00000238009,Gene Expression,GRCh38
AL627309.3,ENSG00000239945,Gene Expression,GRCh38
...,...,...,...
AC141272.1,ENSG00000277836,Gene Expression,GRCh38
AC023491.2,ENSG00000278633,Gene Expression,GRCh38
AC007325.1,ENSG00000276017,Gene Expression,GRCh38
AC007325.4,ENSG00000278817,Gene Expression,GRCh38


In [28]:
#Ensembl IDs

In [29]:
gene_info = pd.read_table('/lustre/scratch127/cellgen/cellgeni/shibla/ref_files/2020A.gene_names.tsv')

In [30]:
gene_info

,ensembl_ids,gene,version
0,ENSG00000243485,MIR1302-2HG,2020A
1,ENSG00000237613,FAM138A,2020A
2,ENSG00000186092,OR4F5,2020A
3,ENSG00000238009,AL627309.1,2020A
4,ENSG00000239945,AL627309.3,2020A
...,...,...,...
36596,ENSG00000277836,AC141272.1,2020A
36597,ENSG00000278633,AC023491.2,2020A
36598,ENSG00000276017,AC007325.1,2020A
36599,ENSG00000278817,AC007325.4,2020A


In [31]:
#create a dictionary with gene symbols and ensembl ids from the gene information file

In [32]:
gene_info_genesym_to_ensembl = dict(zip(gene_info['gene'],gene_info['ensembl_ids']))

In [33]:
gene_info_genesym_to_ensembl

{'MIR1302-2HG': 'ENSG00000243485',
 'FAM138A': 'ENSG00000237613',
 'OR4F5': 'ENSG00000186092',
 'AL627309.1': 'ENSG00000238009',
 'AL627309.3': 'ENSG00000239945',
 'AL627309.2': 'ENSG00000239906',
 'AL627309.5': 'ENSG00000241860',
 'AL627309.4': 'ENSG00000241599',
 'AP006222.2': 'ENSG00000286448',
 'AL732372.1': 'ENSG00000236601',
 'OR4F29': 'ENSG00000284733',
 'AC114498.1': 'ENSG00000235146',
 'OR4F16': 'ENSG00000284662',
 'AL669831.2': 'ENSG00000229905',
 'LINC01409': 'ENSG00000237491',
 'FAM87B': 'ENSG00000177757',
 'LINC01128': 'ENSG00000228794',
 'LINC00115': 'ENSG00000225880',
 'FAM41C': 'ENSG00000230368',
 'AL645608.6': 'ENSG00000272438',
 'AL645608.2': 'ENSG00000230699',
 'AL645608.4': 'ENSG00000241180',
 'LINC02593': 'ENSG00000223764',
 'SAMD11': 'ENSG00000187634',
 'NOC2L': 'ENSG00000188976',
 'KLHL17': 'ENSG00000187961',
 'PLEKHN1': 'ENSG00000187583',
 'PERM1': 'ENSG00000187642',
 'AL645608.7': 'ENSG00000272512',
 'HES4': 'ENSG00000188290',
 'ISG15': 'ENSG00000187608',
 'AL6

In [34]:
#Store ensembl ids in a new column in adata.var by matching gene symbols and ensembl ids from the gene information file

In [35]:
adata.var['gene_symbols'] = adata.var_names

In [36]:
araw.var['gene_symbols'] = araw.var_names

In [37]:
araw.var_names = araw.var['gene_ids']

In [38]:
adata.var['ensembl_id'] = adata.var['gene_symbols'].map(gene_info_genesym_to_ensembl)

In [39]:
adata.var

,gene_symbols,ensembl_id
AL627309.1,AL627309.1,ENSG00000238009
AL627309.5,AL627309.5,ENSG00000241860
LINC01409,LINC01409,ENSG00000237491
LINC01128,LINC01128,ENSG00000228794
LINC00115,LINC00115,ENSG00000225880
...,...,...
MT-CYB,MT-CYB,ENSG00000198727
AC145212.1,AC145212.1,ENSG00000277400
MAFIP,MAFIP,ENSG00000274847
AC007325.4,AC007325.4,ENSG00000278817


In [40]:
adata.var['ensembl_id'].isna().sum()

15

In [41]:
adata.var_names = adata.var['ensembl_id']

In [42]:
adata.var

,gene_symbols,ensembl_id
ensembl_id,,
ENSG00000238009,AL627309.1,ENSG00000238009
ENSG00000241860,AL627309.5,ENSG00000241860
ENSG00000237491,LINC01409,ENSG00000237491
ENSG00000228794,LINC01128,ENSG00000228794
ENSG00000225880,LINC00115,ENSG00000225880
...,...,...
ENSG00000198727,MT-CYB,ENSG00000198727
ENSG00000277400,AC145212.1,ENSG00000277400
ENSG00000274847,MAFIP,ENSG00000274847


In [43]:
del adata.var['gene_symbols']
del adata.var['ensembl_id']

In [44]:
del araw.var['gene_ids']
del araw.var['feature_types']
del araw.var['genome']
del araw.var['gene_symbols']

In [45]:
adata.var.index= adata.var.index.drop_duplicates()

In [46]:
adata.var

""
ensembl_id
ENSG00000238009
ENSG00000241860
ENSG00000237491
ENSG00000228794
ENSG00000225880
...
ENSG00000198727
ENSG00000277400
ENSG00000274847


In [47]:
araw.var

""
gene_ids
ENSG00000243485
ENSG00000237613
ENSG00000186092
ENSG00000238009
ENSG00000239945
...
ENSG00000277836
ENSG00000278633
ENSG00000276017


In [48]:
# load the approved genes file

In [49]:
approved_genes = pd.read_csv('/lustre/scratch127/cellgen/cellgeni/cxgportal_sets/Endometrium_reference_integrated_atlas/genes_approved.csv')

In [50]:
# Create a dictionary from the approved genes file using the symbols and feature id columns.

In [51]:
genedict = {key: 1 for key in list(approved_genes.feature_id)}

In [52]:
genedict

{'ERCC-00002': 1,
 'ERCC-00003': 1,
 'ERCC-00004': 1,
 'ERCC-00009': 1,
 'ERCC-00012': 1,
 'ERCC-00013': 1,
 'ERCC-00014': 1,
 'ERCC-00016': 1,
 'ERCC-00017': 1,
 'ERCC-00019': 1,
 'ERCC-00022': 1,
 'ERCC-00024': 1,
 'ERCC-00025': 1,
 'ERCC-00028': 1,
 'ERCC-00031': 1,
 'ERCC-00033': 1,
 'ERCC-00034': 1,
 'ERCC-00035': 1,
 'ERCC-00039': 1,
 'ERCC-00040': 1,
 'ERCC-00041': 1,
 'ERCC-00042': 1,
 'ERCC-00043': 1,
 'ERCC-00044': 1,
 'ERCC-00046': 1,
 'ERCC-00048': 1,
 'ERCC-00051': 1,
 'ERCC-00053': 1,
 'ERCC-00054': 1,
 'ERCC-00057': 1,
 'ERCC-00058': 1,
 'ERCC-00059': 1,
 'ERCC-00060': 1,
 'ERCC-00061': 1,
 'ERCC-00062': 1,
 'ERCC-00067': 1,
 'ERCC-00069': 1,
 'ERCC-00071': 1,
 'ERCC-00073': 1,
 'ERCC-00074': 1,
 'ERCC-00075': 1,
 'ERCC-00076': 1,
 'ERCC-00077': 1,
 'ERCC-00078': 1,
 'ERCC-00079': 1,
 'ERCC-00081': 1,
 'ERCC-00083': 1,
 'ERCC-00084': 1,
 'ERCC-00085': 1,
 'ERCC-00086': 1,
 'ERCC-00092': 1,
 'ERCC-00095': 1,
 'ERCC-00096': 1,
 'ERCC-00097': 1,
 'ERCC-00098': 1,
 'ERCC-000

In [53]:
len(genedict)

116184

In [54]:
# Filter out the genes which are not in the approved genes file

In [55]:
var_to_keep_adata = [x for x in adata.var_names if (x in genedict)]
var_to_keep_araw = [x for x in araw.var_names if (x in genedict)]

In [56]:
len(var_to_keep_adata)

16992

In [57]:
len(var_to_keep_araw)

36503

In [58]:
# Modify the anndata object by filtering out the filtered genes. copy the index column values to a new column called gene_symbols

In [59]:
adata = adata[:, var_to_keep_adata].copy()
araw = araw[:, var_to_keep_araw].copy()

In [60]:
#  View the var

In [61]:
adata.var

""
ensembl_id
ENSG00000238009
ENSG00000241860
ENSG00000237491
ENSG00000228794
ENSG00000225880
...
ENSG00000198727
ENSG00000277400
ENSG00000274847


In [62]:
araw.var

""
gene_ids
ENSG00000243485
ENSG00000237613
ENSG00000186092
ENSG00000238009
ENSG00000239945
...
ENSG00000277836
ENSG00000278633
ENSG00000276017


feature is filtered

In [63]:
def add_zero():
	global adata
	global araw
	if araw.shape[1] > adata.shape[1]:
		genes_add = [x for x in araw.var.index.to_list() if x not in adata.var.index.to_list()]
		new_matrix = sparse.csr_matrix((adata.X.data, adata.X.indices, adata.X.indptr), shape = araw.shape)
		all_genes = adata.var.index.to_list()
		all_genes.extend(genes_add)
		new_var = pd.DataFrame(index=all_genes)
		new_var = pd.merge(new_var, araw.var, left_index=True, right_index=True, how='left')
		new_var['feature_is_filtered'] = False
		new_var.loc[genes_add, 'feature_is_filtered'] = True
		new_adata = ad.AnnData(X=new_matrix, obs=adata.obs, var=new_var, uns=adata.uns, obsm=adata.obsm)
		if adata.layers:
			for layer in adata.layers:
				new_layer = sparse.csr_matrix((adata.layers[layer].data, adata.layers[layer].indices, adata.layers[layer].indptr), shape = araw.shape)
				new_adata.layers[layer] = new_layer
		new_adata = new_adata[:,araw.var.index.to_list()]
		new_adata.var = new_adata.var.merge(adata.var, left_index=True, right_index=True, how='left')
		adata = new_adata
	else:
		adata.var['feature_is_filtered'] = False

In [64]:
add_zero()

In [65]:
# view var

In [66]:
adata.var

,feature_is_filtered
ENSG00000243485,True
ENSG00000237613,True
ENSG00000186092,True
ENSG00000238009,False
ENSG00000239945,True
...,...
ENSG00000277836,True
ENSG00000278633,True
ENSG00000276017,True
ENSG00000278817,False


In [67]:
list(adata.var['feature_is_filtered'].unique())

[True, False]

In [68]:
True_count = (adata.var['feature_is_filtered']== True).sum()

In [69]:
True_count

19511

In [70]:
araw.var

""
gene_ids
ENSG00000243485
ENSG00000237613
ENSG00000186092
ENSG00000238009
ENSG00000239945
...
ENSG00000277836
ENSG00000278633
ENSG00000276017


#### **Observations(obs) (Cell metadata)**

In [71]:
#view obs

In [72]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_PTGDS+ fibroblasts,c2l_MigDC,c2l_Neuronal_SchwannC,c2l_SMC,c2l_Skeletal muscle cells,leiden,c2l_Suprabasal keratinocytes,c2l_Basal keratinocytes,c2l_Chondrocytes,c2l_IL8+ DC1
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAGCTTTCAGAAG-1,1,43,9,1848,3652,94,2.573932,552,15.115005,AAACAGCTTTCAGAAG-1,...,0.035101,0.074695,0.104958,0.106225,1.857144,3,9.162531,1.033096,0.301104,0.093358
AAACAGGGTCTATATT-1,1,47,13,1877,3577,128,3.578418,537,15.012580,AAACAGGGTCTATATT-1,...,0.046469,0.032088,0.178191,0.115267,0.999841,3,4.930280,0.568820,0.756456,0.043053
AAACCGGGTAGGTACC-1,1,42,28,3439,9152,211,2.305507,1982,21.656469,AAACCGGGTAGGTACC-1,...,0.053109,0.047806,0.272672,0.291519,4.901595,7,27.666287,4.430784,0.575313,0.063929
AAACCGTTCGTCCAGG-1,1,52,42,1403,2472,128,5.177994,310,12.540454,AAACCGTTCGTCCAGG-1,...,0.009489,0.001582,0.088101,0.258719,1.535747,2,2.869617,2.174732,1.126324,0.014893
AAACCTCATGAAGTTG-1,1,37,19,7836,32964,366,1.110302,6706,20.343405,AAACCTCATGAAGTTG-1,...,1.448979,0.620764,1.028491,0.203122,7.427980,6,50.767454,52.277335,0.163867,0.706979
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTGGTAGGAGGGAT-1,1,50,28,953,1536,58,3.776042,294,19.140625,TTGTGGTAGGAGGGAT-1,...,0.002802,0.005221,0.044680,0.033479,0.807696,3,4.759240,0.566145,0.102280,0.009124
TTGTGTATGCCACCAA-1,1,56,60,589,797,67,8.406525,92,11.543287,TTGTGTATGCCACCAA-1,...,0.002312,0.000742,0.046971,0.039538,0.450428,1,0.263667,0.144377,0.218212,0.000867
TTGTGTTTCCCGAAAG-1,1,51,59,939,1411,80,5.669738,174,12.331679,TTGTGTTTCCCGAAAG-1,...,0.009261,0.001219,0.070533,0.105023,0.397123,1,0.583944,0.659881,0.599904,0.007331


In [73]:
adata.obs.columns

Index(['in_tissue', 'array_row', 'array_col', 'n_genes_by_counts',
       'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb',
       'pct_counts_rb', 'barcode', 'c2l_Th', 'c2l_NK', 'c2l_APOD+ fibroblasts',
       'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2', 'c2l_DC1',
       'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes',
       'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc',
       'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes',
       'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts',
       'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC',
       'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes',
       'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1'],
      dtype='object')

#### **assay_ontology_term_id**

In [74]:
# add the assay_ontology_term_id column

In [75]:
adata.obs['assay_ontology_term_id'] = ['EFO:0010961'] * len(adata.obs)

In [76]:
# change datatype of the column

In [77]:
adata.obs['assay_ontology_term_id'] = adata.obs['assay_ontology_term_id'].astype('category')

In [78]:
# view adata.obs

In [79]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_MigDC,c2l_Neuronal_SchwannC,c2l_SMC,c2l_Skeletal muscle cells,leiden,c2l_Suprabasal keratinocytes,c2l_Basal keratinocytes,c2l_Chondrocytes,c2l_IL8+ DC1,assay_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAGCTTTCAGAAG-1,1,43,9,1848,3652,94,2.573932,552,15.115005,AAACAGCTTTCAGAAG-1,...,0.074695,0.104958,0.106225,1.857144,3,9.162531,1.033096,0.301104,0.093358,EFO:0010961
AAACAGGGTCTATATT-1,1,47,13,1877,3577,128,3.578418,537,15.012580,AAACAGGGTCTATATT-1,...,0.032088,0.178191,0.115267,0.999841,3,4.930280,0.568820,0.756456,0.043053,EFO:0010961
AAACCGGGTAGGTACC-1,1,42,28,3439,9152,211,2.305507,1982,21.656469,AAACCGGGTAGGTACC-1,...,0.047806,0.272672,0.291519,4.901595,7,27.666287,4.430784,0.575313,0.063929,EFO:0010961
AAACCGTTCGTCCAGG-1,1,52,42,1403,2472,128,5.177994,310,12.540454,AAACCGTTCGTCCAGG-1,...,0.001582,0.088101,0.258719,1.535747,2,2.869617,2.174732,1.126324,0.014893,EFO:0010961
AAACCTCATGAAGTTG-1,1,37,19,7836,32964,366,1.110302,6706,20.343405,AAACCTCATGAAGTTG-1,...,0.620764,1.028491,0.203122,7.427980,6,50.767454,52.277335,0.163867,0.706979,EFO:0010961
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTGGTAGGAGGGAT-1,1,50,28,953,1536,58,3.776042,294,19.140625,TTGTGGTAGGAGGGAT-1,...,0.005221,0.044680,0.033479,0.807696,3,4.759240,0.566145,0.102280,0.009124,EFO:0010961
TTGTGTATGCCACCAA-1,1,56,60,589,797,67,8.406525,92,11.543287,TTGTGTATGCCACCAA-1,...,0.000742,0.046971,0.039538,0.450428,1,0.263667,0.144377,0.218212,0.000867,EFO:0010961
TTGTGTTTCCCGAAAG-1,1,51,59,939,1411,80,5.669738,174,12.331679,TTGTGTTTCCCGAAAG-1,...,0.001219,0.070533,0.105023,0.397123,1,0.583944,0.659881,0.599904,0.007331,EFO:0010961


#### **cell_type_ontology_term_id**

In [80]:
#get the column in adata.obs related. to cell type annotation

In [81]:
adata.obs.columns

Index(['in_tissue', 'array_row', 'array_col', 'n_genes_by_counts',
       'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb',
       'pct_counts_rb', 'barcode', 'c2l_Th', 'c2l_NK', 'c2l_APOD+ fibroblasts',
       'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2', 'c2l_DC1',
       'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes',
       'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc',
       'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes',
       'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts',
       'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC',
       'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes',
       'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1',
       'assay_ontology_term_id'],
      dtype='object')

In [82]:
adata.obsm

AxisArrays with keys: X_spatial

In [83]:
c2l_columns = [col for col in adata.obs.columns if col.startswith('c2l')]

In [84]:
c2l_columns

['c2l_Th',
 'c2l_NK',
 'c2l_APOD+ fibroblasts',
 'c2l_CD8+ T RM',
 'c2l_T reg',
 'c2l_Macro1_2',
 'c2l_DC1',
 'c2l_SFRP2+ fibroblasts',
 'c2l_TAGLN+ pericytes',
 'c2l_POSTN+ fibroblasts',
 'c2l_RGS5+ pericytes',
 'c2l_VEC',
 'c2l_Tc',
 'c2l_ILC_NK',
 'c2l_BC',
 'c2l_Monocytes',
 'c2l_MastC',
 'c2l_Melanocytes',
 'c2l_DC2',
 'c2l_LEC',
 'c2l_PlasmaC',
 'c2l_PTGDS+ fibroblasts',
 'c2l_MigDC',
 'c2l_Neuronal_SchwannC',
 'c2l_SMC',
 'c2l_Skeletal muscle cells',
 'c2l_Suprabasal keratinocytes',
 'c2l_Basal keratinocytes',
 'c2l_Chondrocytes',
 'c2l_IL8+ DC1']

In [85]:
adata.obs['max_c2l_column'] = adata.obs[c2l_columns].idxmax(axis=1)

In [86]:
adata.obs['max_c2l_column_value'] = adata.obs[c2l_columns].max(axis=1)

In [87]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_SMC,c2l_Skeletal muscle cells,leiden,c2l_Suprabasal keratinocytes,c2l_Basal keratinocytes,c2l_Chondrocytes,c2l_IL8+ DC1,assay_ontology_term_id,max_c2l_column,max_c2l_column_value
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAGCTTTCAGAAG-1,1,43,9,1848,3652,94,2.573932,552,15.115005,AAACAGCTTTCAGAAG-1,...,0.106225,1.857144,3,9.162531,1.033096,0.301104,0.093358,EFO:0010961,c2l_Suprabasal keratinocytes,9.162531
AAACAGGGTCTATATT-1,1,47,13,1877,3577,128,3.578418,537,15.012580,AAACAGGGTCTATATT-1,...,0.115267,0.999841,3,4.930280,0.568820,0.756456,0.043053,EFO:0010961,c2l_Suprabasal keratinocytes,4.930280
AAACCGGGTAGGTACC-1,1,42,28,3439,9152,211,2.305507,1982,21.656469,AAACCGGGTAGGTACC-1,...,0.291519,4.901595,7,27.666287,4.430784,0.575313,0.063929,EFO:0010961,c2l_Suprabasal keratinocytes,27.666287
AAACCGTTCGTCCAGG-1,1,52,42,1403,2472,128,5.177994,310,12.540454,AAACCGTTCGTCCAGG-1,...,0.258719,1.535747,2,2.869617,2.174732,1.126324,0.014893,EFO:0010961,c2l_SFRP2+ fibroblasts,4.678117
AAACCTCATGAAGTTG-1,1,37,19,7836,32964,366,1.110302,6706,20.343405,AAACCTCATGAAGTTG-1,...,0.203122,7.427980,6,50.767454,52.277335,0.163867,0.706979,EFO:0010961,c2l_Basal keratinocytes,52.277335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTGGTAGGAGGGAT-1,1,50,28,953,1536,58,3.776042,294,19.140625,TTGTGGTAGGAGGGAT-1,...,0.033479,0.807696,3,4.759240,0.566145,0.102280,0.009124,EFO:0010961,c2l_Suprabasal keratinocytes,4.759240
TTGTGTATGCCACCAA-1,1,56,60,589,797,67,8.406525,92,11.543287,TTGTGTATGCCACCAA-1,...,0.039538,0.450428,1,0.263667,0.144377,0.218212,0.000867,EFO:0010961,c2l_VEC,1.141842
TTGTGTTTCCCGAAAG-1,1,51,59,939,1411,80,5.669738,174,12.331679,TTGTGTTTCCCGAAAG-1,...,0.105023,0.397123,1,0.583944,0.659881,0.599904,0.007331,EFO:0010961,c2l_SFRP2+ fibroblasts,1.983099


In [88]:
mapping= {'c2l_Th' :'CL:0000912',
 'c2l_NK':'CL:0000623',
 'c2l_APOD+ fibroblasts':'CL:0000057',
 'c2l_CD8+ T RM':'CL:0001203' ,
 'c2l_T reg':'CL:0000815',
 'c2l_Macro1_2':'CL:0000235',
 'c2l_DC1':'CL:0000990',
 'c2l_SFRP2+ fibroblasts':'CL:0000057',
 'c2l_TAGLN+ pericytes':'CL:0000669',
 'c2l_POSTN+ fibroblasts':'CL:0000057',
 'c2l_RGS5+ pericytes':'CL:0000669',
 'c2l_VEC':'CL:0002139',
 'c2l_Tc':'CL:0000910',
 'c2l_ILC_NK':'CL:0001065', #not NK
 'c2l_BC':'CL:0000646',
 'c2l_Monocytes':'CL:0000576',
 'c2l_MastC':'CL:0000097',
 'c2l_Melanocytes':'CL:1000458',
 'c2l_DC2':'CL:0000784',
 'c2l_LEC':'CL:0002138',
 'c2l_PlasmaC':'CL:0000786',
 'c2l_PTGDS+ fibroblasts':'CL:0000057',
 'c2l_MigDC':'CL:0000451', #not mig
 'c2l_Neuronal_SchwannC':'CL:0002573',
 'c2l_SMC':'CL:0000192',
 'c2l_Skeletal muscle cells':'CL:0000188',
 'c2l_Suprabasal keratinocytes':'CL:4033013',
 'c2l_Basal keratinocytes':'CL:0002187',
 'c2l_Chondrocytes':'CL:0000138',
 'c2l_IL8+ DC1':'CL:0000990'}

In [89]:
# create a dictionary of cell type and ontology term

In [90]:
# add the cell_type_ontology_term_id column

In [91]:
adata.obs['cell_type_ontology_term_id'] = adata.obs['max_c2l_column'].map(mapping)

In [92]:
# change datatype of the column

In [93]:
adata.obs['cell_type_ontology_term_id'] = adata.obs['cell_type_ontology_term_id'].astype('category')

In [94]:
list(adata.obs['cell_type_ontology_term_id'].unique())

['CL:4033013',
 'CL:0000057',
 'CL:0002187',
 'CL:0002139',
 'CL:0000910',
 'CL:0000138',
 'CL:0000623',
 'CL:0000669',
 'CL:0002138',
 'CL:0000646',
 'CL:0000192',
 'CL:0000576',
 'CL:0000188',
 'CL:0000815']

In [95]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_Skeletal muscle cells,leiden,c2l_Suprabasal keratinocytes,c2l_Basal keratinocytes,c2l_Chondrocytes,c2l_IL8+ DC1,assay_ontology_term_id,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAGCTTTCAGAAG-1,1,43,9,1848,3652,94,2.573932,552,15.115005,AAACAGCTTTCAGAAG-1,...,1.857144,3,9.162531,1.033096,0.301104,0.093358,EFO:0010961,c2l_Suprabasal keratinocytes,9.162531,CL:4033013
AAACAGGGTCTATATT-1,1,47,13,1877,3577,128,3.578418,537,15.012580,AAACAGGGTCTATATT-1,...,0.999841,3,4.930280,0.568820,0.756456,0.043053,EFO:0010961,c2l_Suprabasal keratinocytes,4.930280,CL:4033013
AAACCGGGTAGGTACC-1,1,42,28,3439,9152,211,2.305507,1982,21.656469,AAACCGGGTAGGTACC-1,...,4.901595,7,27.666287,4.430784,0.575313,0.063929,EFO:0010961,c2l_Suprabasal keratinocytes,27.666287,CL:4033013
AAACCGTTCGTCCAGG-1,1,52,42,1403,2472,128,5.177994,310,12.540454,AAACCGTTCGTCCAGG-1,...,1.535747,2,2.869617,2.174732,1.126324,0.014893,EFO:0010961,c2l_SFRP2+ fibroblasts,4.678117,CL:0000057
AAACCTCATGAAGTTG-1,1,37,19,7836,32964,366,1.110302,6706,20.343405,AAACCTCATGAAGTTG-1,...,7.427980,6,50.767454,52.277335,0.163867,0.706979,EFO:0010961,c2l_Basal keratinocytes,52.277335,CL:0002187
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTGGTAGGAGGGAT-1,1,50,28,953,1536,58,3.776042,294,19.140625,TTGTGGTAGGAGGGAT-1,...,0.807696,3,4.759240,0.566145,0.102280,0.009124,EFO:0010961,c2l_Suprabasal keratinocytes,4.759240,CL:4033013
TTGTGTATGCCACCAA-1,1,56,60,589,797,67,8.406525,92,11.543287,TTGTGTATGCCACCAA-1,...,0.450428,1,0.263667,0.144377,0.218212,0.000867,EFO:0010961,c2l_VEC,1.141842,CL:0002139
TTGTGTTTCCCGAAAG-1,1,51,59,939,1411,80,5.669738,174,12.331679,TTGTGTTTCCCGAAAG-1,...,0.397123,1,0.583944,0.659881,0.599904,0.007331,EFO:0010961,c2l_SFRP2+ fibroblasts,1.983099,CL:0000057


In [96]:
araw.obs

""
AAACAGCTTTCAGAAG-1
AAACAGGGTCTATATT-1
AAACCGGGTAGGTACC-1
AAACCGTTCGTCCAGG-1
AAACCTCATGAAGTTG-1
...
TTGTGGTAGGAGGGAT-1
TTGTGTATGCCACCAA-1
TTGTGTTTCCCGAAAG-1
TTGTTTCATTAGTCTA-1


#### **donor_id**

In [97]:
#identify the column in adata.obs which provides donor information

In [98]:
adata.obs.columns

Index(['in_tissue', 'array_row', 'array_col', 'n_genes_by_counts',
       'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb',
       'pct_counts_rb', 'barcode', 'c2l_Th', 'c2l_NK', 'c2l_APOD+ fibroblasts',
       'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2', 'c2l_DC1',
       'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes',
       'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc',
       'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes',
       'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts',
       'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC',
       'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes',
       'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1',
       'assay_ontology_term_id', 'max_c2l_column', 'max_c2l_column_value',
       'cell_type_ontology_term_id'],
      dtype='object')

In [99]:
# add the donor_id column

In [100]:
adata.obs['donor_id'] = ['face_forehead_A'] * len(adata.obs)

In [101]:
# change datatype of the column

In [102]:
adata.obs['donor_id'] = adata.obs['donor_id'].astype('category')

In [103]:
# view unique values of donor_id column

In [104]:
list(adata.obs['donor_id'].unique())

['face_forehead1']

In [105]:
#view obs

In [106]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,leiden,c2l_Suprabasal keratinocytes,c2l_Basal keratinocytes,c2l_Chondrocytes,c2l_IL8+ DC1,assay_ontology_term_id,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id,donor_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAGCTTTCAGAAG-1,1,43,9,1848,3652,94,2.573932,552,15.115005,AAACAGCTTTCAGAAG-1,...,3,9.162531,1.033096,0.301104,0.093358,EFO:0010961,c2l_Suprabasal keratinocytes,9.162531,CL:4033013,face_forehead1
AAACAGGGTCTATATT-1,1,47,13,1877,3577,128,3.578418,537,15.012580,AAACAGGGTCTATATT-1,...,3,4.930280,0.568820,0.756456,0.043053,EFO:0010961,c2l_Suprabasal keratinocytes,4.930280,CL:4033013,face_forehead1
AAACCGGGTAGGTACC-1,1,42,28,3439,9152,211,2.305507,1982,21.656469,AAACCGGGTAGGTACC-1,...,7,27.666287,4.430784,0.575313,0.063929,EFO:0010961,c2l_Suprabasal keratinocytes,27.666287,CL:4033013,face_forehead1
AAACCGTTCGTCCAGG-1,1,52,42,1403,2472,128,5.177994,310,12.540454,AAACCGTTCGTCCAGG-1,...,2,2.869617,2.174732,1.126324,0.014893,EFO:0010961,c2l_SFRP2+ fibroblasts,4.678117,CL:0000057,face_forehead1
AAACCTCATGAAGTTG-1,1,37,19,7836,32964,366,1.110302,6706,20.343405,AAACCTCATGAAGTTG-1,...,6,50.767454,52.277335,0.163867,0.706979,EFO:0010961,c2l_Basal keratinocytes,52.277335,CL:0002187,face_forehead1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTGGTAGGAGGGAT-1,1,50,28,953,1536,58,3.776042,294,19.140625,TTGTGGTAGGAGGGAT-1,...,3,4.759240,0.566145,0.102280,0.009124,EFO:0010961,c2l_Suprabasal keratinocytes,4.759240,CL:4033013,face_forehead1
TTGTGTATGCCACCAA-1,1,56,60,589,797,67,8.406525,92,11.543287,TTGTGTATGCCACCAA-1,...,1,0.263667,0.144377,0.218212,0.000867,EFO:0010961,c2l_VEC,1.141842,CL:0002139,face_forehead1
TTGTGTTTCCCGAAAG-1,1,51,59,939,1411,80,5.669738,174,12.331679,TTGTGTTTCCCGAAAG-1,...,1,0.583944,0.659881,0.599904,0.007331,EFO:0010961,c2l_SFRP2+ fibroblasts,1.983099,CL:0000057,face_forehead1


In [107]:
adata.obs.columns

Index(['in_tissue', 'array_row', 'array_col', 'n_genes_by_counts',
       'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb',
       'pct_counts_rb', 'barcode', 'c2l_Th', 'c2l_NK', 'c2l_APOD+ fibroblasts',
       'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2', 'c2l_DC1',
       'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes',
       'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc',
       'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes',
       'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts',
       'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC',
       'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes',
       'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1',
       'assay_ontology_term_id', 'max_c2l_column', 'max_c2l_column_value',
       'cell_type_ontology_term_id', 'donor_id'],
      dtype='object')

#### **development_stage_ontology_term_id**

In [108]:
# identify the column in adata which corresponds to age

In [109]:
# add the development_stage_ontology_term_id column

In [110]:
adata.obs['development_stage_ontology_term_id'] = ['HsapDv:0000149'] * len(adata.obs)

In [111]:
# change datatype of the column

In [112]:
adata.obs['development_stage_ontology_term_id'] = adata.obs['development_stage_ontology_term_id'].astype('category')

In [113]:
# view unique values of development_stage_ontology_term_id column

In [114]:
list(adata.obs['development_stage_ontology_term_id'].unique())

['HsapDv:0000149']

In [115]:
# view adata.obs

In [116]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_Suprabasal keratinocytes,c2l_Basal keratinocytes,c2l_Chondrocytes,c2l_IL8+ DC1,assay_ontology_term_id,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAGCTTTCAGAAG-1,1,43,9,1848,3652,94,2.573932,552,15.115005,AAACAGCTTTCAGAAG-1,...,9.162531,1.033096,0.301104,0.093358,EFO:0010961,c2l_Suprabasal keratinocytes,9.162531,CL:4033013,face_forehead1,HsapDv:0000149
AAACAGGGTCTATATT-1,1,47,13,1877,3577,128,3.578418,537,15.012580,AAACAGGGTCTATATT-1,...,4.930280,0.568820,0.756456,0.043053,EFO:0010961,c2l_Suprabasal keratinocytes,4.930280,CL:4033013,face_forehead1,HsapDv:0000149
AAACCGGGTAGGTACC-1,1,42,28,3439,9152,211,2.305507,1982,21.656469,AAACCGGGTAGGTACC-1,...,27.666287,4.430784,0.575313,0.063929,EFO:0010961,c2l_Suprabasal keratinocytes,27.666287,CL:4033013,face_forehead1,HsapDv:0000149
AAACCGTTCGTCCAGG-1,1,52,42,1403,2472,128,5.177994,310,12.540454,AAACCGTTCGTCCAGG-1,...,2.869617,2.174732,1.126324,0.014893,EFO:0010961,c2l_SFRP2+ fibroblasts,4.678117,CL:0000057,face_forehead1,HsapDv:0000149
AAACCTCATGAAGTTG-1,1,37,19,7836,32964,366,1.110302,6706,20.343405,AAACCTCATGAAGTTG-1,...,50.767454,52.277335,0.163867,0.706979,EFO:0010961,c2l_Basal keratinocytes,52.277335,CL:0002187,face_forehead1,HsapDv:0000149
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTGGTAGGAGGGAT-1,1,50,28,953,1536,58,3.776042,294,19.140625,TTGTGGTAGGAGGGAT-1,...,4.759240,0.566145,0.102280,0.009124,EFO:0010961,c2l_Suprabasal keratinocytes,4.759240,CL:4033013,face_forehead1,HsapDv:0000149
TTGTGTATGCCACCAA-1,1,56,60,589,797,67,8.406525,92,11.543287,TTGTGTATGCCACCAA-1,...,0.263667,0.144377,0.218212,0.000867,EFO:0010961,c2l_VEC,1.141842,CL:0002139,face_forehead1,HsapDv:0000149
TTGTGTTTCCCGAAAG-1,1,51,59,939,1411,80,5.669738,174,12.331679,TTGTGTTTCCCGAAAG-1,...,0.583944,0.659881,0.599904,0.007331,EFO:0010961,c2l_SFRP2+ fibroblasts,1.983099,CL:0000057,face_forehead1,HsapDv:0000149


#### **disease_ontology_term_id**

In [117]:
# Assign normal since all are healthy patients

In [118]:
# add the disease_ontology_term_id column

In [119]:
adata.obs['disease_ontology_term_id'] = ['PATO:0000461'] * len(adata.obs)

In [120]:
#change data type of column

In [121]:
adata.obs['disease_ontology_term_id'] = adata.obs['disease_ontology_term_id'].astype('category')

In [122]:
# view obs

In [123]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_Basal keratinocytes,c2l_Chondrocytes,c2l_IL8+ DC1,assay_ontology_term_id,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAGCTTTCAGAAG-1,1,43,9,1848,3652,94,2.573932,552,15.115005,AAACAGCTTTCAGAAG-1,...,1.033096,0.301104,0.093358,EFO:0010961,c2l_Suprabasal keratinocytes,9.162531,CL:4033013,face_forehead1,HsapDv:0000149,PATO:0000461
AAACAGGGTCTATATT-1,1,47,13,1877,3577,128,3.578418,537,15.012580,AAACAGGGTCTATATT-1,...,0.568820,0.756456,0.043053,EFO:0010961,c2l_Suprabasal keratinocytes,4.930280,CL:4033013,face_forehead1,HsapDv:0000149,PATO:0000461
AAACCGGGTAGGTACC-1,1,42,28,3439,9152,211,2.305507,1982,21.656469,AAACCGGGTAGGTACC-1,...,4.430784,0.575313,0.063929,EFO:0010961,c2l_Suprabasal keratinocytes,27.666287,CL:4033013,face_forehead1,HsapDv:0000149,PATO:0000461
AAACCGTTCGTCCAGG-1,1,52,42,1403,2472,128,5.177994,310,12.540454,AAACCGTTCGTCCAGG-1,...,2.174732,1.126324,0.014893,EFO:0010961,c2l_SFRP2+ fibroblasts,4.678117,CL:0000057,face_forehead1,HsapDv:0000149,PATO:0000461
AAACCTCATGAAGTTG-1,1,37,19,7836,32964,366,1.110302,6706,20.343405,AAACCTCATGAAGTTG-1,...,52.277335,0.163867,0.706979,EFO:0010961,c2l_Basal keratinocytes,52.277335,CL:0002187,face_forehead1,HsapDv:0000149,PATO:0000461
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTGGTAGGAGGGAT-1,1,50,28,953,1536,58,3.776042,294,19.140625,TTGTGGTAGGAGGGAT-1,...,0.566145,0.102280,0.009124,EFO:0010961,c2l_Suprabasal keratinocytes,4.759240,CL:4033013,face_forehead1,HsapDv:0000149,PATO:0000461
TTGTGTATGCCACCAA-1,1,56,60,589,797,67,8.406525,92,11.543287,TTGTGTATGCCACCAA-1,...,0.144377,0.218212,0.000867,EFO:0010961,c2l_VEC,1.141842,CL:0002139,face_forehead1,HsapDv:0000149,PATO:0000461
TTGTGTTTCCCGAAAG-1,1,51,59,939,1411,80,5.669738,174,12.331679,TTGTGTTTCCCGAAAG-1,...,0.659881,0.599904,0.007331,EFO:0010961,c2l_SFRP2+ fibroblasts,1.983099,CL:0000057,face_forehead1,HsapDv:0000149,PATO:0000461


#### **is_primary_data**

In [124]:
adata.obs['is_primary_data'] = [True] * len(adata.obs)

In [125]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_Chondrocytes,c2l_IL8+ DC1,assay_ontology_term_id,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAGCTTTCAGAAG-1,1,43,9,1848,3652,94,2.573932,552,15.115005,AAACAGCTTTCAGAAG-1,...,0.301104,0.093358,EFO:0010961,c2l_Suprabasal keratinocytes,9.162531,CL:4033013,face_forehead1,HsapDv:0000149,PATO:0000461,True
AAACAGGGTCTATATT-1,1,47,13,1877,3577,128,3.578418,537,15.012580,AAACAGGGTCTATATT-1,...,0.756456,0.043053,EFO:0010961,c2l_Suprabasal keratinocytes,4.930280,CL:4033013,face_forehead1,HsapDv:0000149,PATO:0000461,True
AAACCGGGTAGGTACC-1,1,42,28,3439,9152,211,2.305507,1982,21.656469,AAACCGGGTAGGTACC-1,...,0.575313,0.063929,EFO:0010961,c2l_Suprabasal keratinocytes,27.666287,CL:4033013,face_forehead1,HsapDv:0000149,PATO:0000461,True
AAACCGTTCGTCCAGG-1,1,52,42,1403,2472,128,5.177994,310,12.540454,AAACCGTTCGTCCAGG-1,...,1.126324,0.014893,EFO:0010961,c2l_SFRP2+ fibroblasts,4.678117,CL:0000057,face_forehead1,HsapDv:0000149,PATO:0000461,True
AAACCTCATGAAGTTG-1,1,37,19,7836,32964,366,1.110302,6706,20.343405,AAACCTCATGAAGTTG-1,...,0.163867,0.706979,EFO:0010961,c2l_Basal keratinocytes,52.277335,CL:0002187,face_forehead1,HsapDv:0000149,PATO:0000461,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTGGTAGGAGGGAT-1,1,50,28,953,1536,58,3.776042,294,19.140625,TTGTGGTAGGAGGGAT-1,...,0.102280,0.009124,EFO:0010961,c2l_Suprabasal keratinocytes,4.759240,CL:4033013,face_forehead1,HsapDv:0000149,PATO:0000461,True
TTGTGTATGCCACCAA-1,1,56,60,589,797,67,8.406525,92,11.543287,TTGTGTATGCCACCAA-1,...,0.218212,0.000867,EFO:0010961,c2l_VEC,1.141842,CL:0002139,face_forehead1,HsapDv:0000149,PATO:0000461,True
TTGTGTTTCCCGAAAG-1,1,51,59,939,1411,80,5.669738,174,12.331679,TTGTGTTTCCCGAAAG-1,...,0.599904,0.007331,EFO:0010961,c2l_SFRP2+ fibroblasts,1.983099,CL:0000057,face_forehead1,HsapDv:0000149,PATO:0000461,True


In [126]:
#change data type of column

In [127]:
adata.obs['is_primary_data'] = adata.obs['is_primary_data'].astype('bool')

#### **organism_ontology_term_id**

In [128]:
# assign organism id 

In [129]:
adata.obs['organism_ontology_term_id'] = ['NCBITaxon:9606'] * len(adata.obs)

In [130]:
#change data type of column

In [131]:
adata.obs['organism_ontology_term_id'] = adata.obs['organism_ontology_term_id'].astype('category')

In [132]:
# view obs

In [133]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_IL8+ DC1,assay_ontology_term_id,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data,organism_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAGCTTTCAGAAG-1,1,43,9,1848,3652,94,2.573932,552,15.115005,AAACAGCTTTCAGAAG-1,...,0.093358,EFO:0010961,c2l_Suprabasal keratinocytes,9.162531,CL:4033013,face_forehead1,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606
AAACAGGGTCTATATT-1,1,47,13,1877,3577,128,3.578418,537,15.012580,AAACAGGGTCTATATT-1,...,0.043053,EFO:0010961,c2l_Suprabasal keratinocytes,4.930280,CL:4033013,face_forehead1,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606
AAACCGGGTAGGTACC-1,1,42,28,3439,9152,211,2.305507,1982,21.656469,AAACCGGGTAGGTACC-1,...,0.063929,EFO:0010961,c2l_Suprabasal keratinocytes,27.666287,CL:4033013,face_forehead1,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606
AAACCGTTCGTCCAGG-1,1,52,42,1403,2472,128,5.177994,310,12.540454,AAACCGTTCGTCCAGG-1,...,0.014893,EFO:0010961,c2l_SFRP2+ fibroblasts,4.678117,CL:0000057,face_forehead1,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606
AAACCTCATGAAGTTG-1,1,37,19,7836,32964,366,1.110302,6706,20.343405,AAACCTCATGAAGTTG-1,...,0.706979,EFO:0010961,c2l_Basal keratinocytes,52.277335,CL:0002187,face_forehead1,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTGGTAGGAGGGAT-1,1,50,28,953,1536,58,3.776042,294,19.140625,TTGTGGTAGGAGGGAT-1,...,0.009124,EFO:0010961,c2l_Suprabasal keratinocytes,4.759240,CL:4033013,face_forehead1,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606
TTGTGTATGCCACCAA-1,1,56,60,589,797,67,8.406525,92,11.543287,TTGTGTATGCCACCAA-1,...,0.000867,EFO:0010961,c2l_VEC,1.141842,CL:0002139,face_forehead1,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606
TTGTGTTTCCCGAAAG-1,1,51,59,939,1411,80,5.669738,174,12.331679,TTGTGTTTCCCGAAAG-1,...,0.007331,EFO:0010961,c2l_SFRP2+ fibroblasts,1.983099,CL:0000057,face_forehead1,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606


#### **self_reported_ethnicity_ontology_term_id**

In [134]:
adata.obs['self_reported_ethnicity_ontology_term_id'] = ['unknown'] * len(adata.obs)

In [135]:
# change data type

In [136]:
adata.obs['self_reported_ethnicity_ontology_term_id'] = adata.obs['self_reported_ethnicity_ontology_term_id'].astype('category')

In [137]:
# view obs

In [138]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,assay_ontology_term_id,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data,organism_ontology_term_id,self_reported_ethnicity_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAGCTTTCAGAAG-1,1,43,9,1848,3652,94,2.573932,552,15.115005,AAACAGCTTTCAGAAG-1,...,EFO:0010961,c2l_Suprabasal keratinocytes,9.162531,CL:4033013,face_forehead1,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown
AAACAGGGTCTATATT-1,1,47,13,1877,3577,128,3.578418,537,15.012580,AAACAGGGTCTATATT-1,...,EFO:0010961,c2l_Suprabasal keratinocytes,4.930280,CL:4033013,face_forehead1,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown
AAACCGGGTAGGTACC-1,1,42,28,3439,9152,211,2.305507,1982,21.656469,AAACCGGGTAGGTACC-1,...,EFO:0010961,c2l_Suprabasal keratinocytes,27.666287,CL:4033013,face_forehead1,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown
AAACCGTTCGTCCAGG-1,1,52,42,1403,2472,128,5.177994,310,12.540454,AAACCGTTCGTCCAGG-1,...,EFO:0010961,c2l_SFRP2+ fibroblasts,4.678117,CL:0000057,face_forehead1,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown
AAACCTCATGAAGTTG-1,1,37,19,7836,32964,366,1.110302,6706,20.343405,AAACCTCATGAAGTTG-1,...,EFO:0010961,c2l_Basal keratinocytes,52.277335,CL:0002187,face_forehead1,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTGGTAGGAGGGAT-1,1,50,28,953,1536,58,3.776042,294,19.140625,TTGTGGTAGGAGGGAT-1,...,EFO:0010961,c2l_Suprabasal keratinocytes,4.759240,CL:4033013,face_forehead1,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown
TTGTGTATGCCACCAA-1,1,56,60,589,797,67,8.406525,92,11.543287,TTGTGTATGCCACCAA-1,...,EFO:0010961,c2l_VEC,1.141842,CL:0002139,face_forehead1,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown
TTGTGTTTCCCGAAAG-1,1,51,59,939,1411,80,5.669738,174,12.331679,TTGTGTTTCCCGAAAG-1,...,EFO:0010961,c2l_SFRP2+ fibroblasts,1.983099,CL:0000057,face_forehead1,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown


In [139]:
list(adata.obs['self_reported_ethnicity_ontology_term_id'].unique())

['unknown']

#### **sex_ontology_term_id**

In [140]:
adata.obs['sex_ontology_term_id'] = ['PATO:0000384'] * len(adata.obs)

In [141]:
# change data type

In [142]:
adata.obs['sex_ontology_term_id'] = adata.obs['sex_ontology_term_id'].astype('category')

In [143]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data,organism_ontology_term_id,self_reported_ethnicity_ontology_term_id,sex_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAGCTTTCAGAAG-1,1,43,9,1848,3652,94,2.573932,552,15.115005,AAACAGCTTTCAGAAG-1,...,c2l_Suprabasal keratinocytes,9.162531,CL:4033013,face_forehead1,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384
AAACAGGGTCTATATT-1,1,47,13,1877,3577,128,3.578418,537,15.012580,AAACAGGGTCTATATT-1,...,c2l_Suprabasal keratinocytes,4.930280,CL:4033013,face_forehead1,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384
AAACCGGGTAGGTACC-1,1,42,28,3439,9152,211,2.305507,1982,21.656469,AAACCGGGTAGGTACC-1,...,c2l_Suprabasal keratinocytes,27.666287,CL:4033013,face_forehead1,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384
AAACCGTTCGTCCAGG-1,1,52,42,1403,2472,128,5.177994,310,12.540454,AAACCGTTCGTCCAGG-1,...,c2l_SFRP2+ fibroblasts,4.678117,CL:0000057,face_forehead1,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384
AAACCTCATGAAGTTG-1,1,37,19,7836,32964,366,1.110302,6706,20.343405,AAACCTCATGAAGTTG-1,...,c2l_Basal keratinocytes,52.277335,CL:0002187,face_forehead1,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTGGTAGGAGGGAT-1,1,50,28,953,1536,58,3.776042,294,19.140625,TTGTGGTAGGAGGGAT-1,...,c2l_Suprabasal keratinocytes,4.759240,CL:4033013,face_forehead1,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384
TTGTGTATGCCACCAA-1,1,56,60,589,797,67,8.406525,92,11.543287,TTGTGTATGCCACCAA-1,...,c2l_VEC,1.141842,CL:0002139,face_forehead1,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384
TTGTGTTTCCCGAAAG-1,1,51,59,939,1411,80,5.669738,174,12.331679,TTGTGTTTCCCGAAAG-1,...,c2l_SFRP2+ fibroblasts,1.983099,CL:0000057,face_forehead1,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384


#### **suspension_type**

In [144]:
# since visium suspension type is 'na'

In [145]:
adata.obs['suspension_type'] = ['na'] * len(adata.obs)

In [146]:
# change data type

In [147]:
adata.obs['suspension_type'] = adata.obs['suspension_type'].astype('category')

In [148]:
# view obs

In [149]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,max_c2l_column_value,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data,organism_ontology_term_id,self_reported_ethnicity_ontology_term_id,sex_ontology_term_id,suspension_type
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAGCTTTCAGAAG-1,1,43,9,1848,3652,94,2.573932,552,15.115005,AAACAGCTTTCAGAAG-1,...,9.162531,CL:4033013,face_forehead1,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na
AAACAGGGTCTATATT-1,1,47,13,1877,3577,128,3.578418,537,15.012580,AAACAGGGTCTATATT-1,...,4.930280,CL:4033013,face_forehead1,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na
AAACCGGGTAGGTACC-1,1,42,28,3439,9152,211,2.305507,1982,21.656469,AAACCGGGTAGGTACC-1,...,27.666287,CL:4033013,face_forehead1,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na
AAACCGTTCGTCCAGG-1,1,52,42,1403,2472,128,5.177994,310,12.540454,AAACCGTTCGTCCAGG-1,...,4.678117,CL:0000057,face_forehead1,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na
AAACCTCATGAAGTTG-1,1,37,19,7836,32964,366,1.110302,6706,20.343405,AAACCTCATGAAGTTG-1,...,52.277335,CL:0002187,face_forehead1,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTGGTAGGAGGGAT-1,1,50,28,953,1536,58,3.776042,294,19.140625,TTGTGGTAGGAGGGAT-1,...,4.759240,CL:4033013,face_forehead1,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na
TTGTGTATGCCACCAA-1,1,56,60,589,797,67,8.406525,92,11.543287,TTGTGTATGCCACCAA-1,...,1.141842,CL:0002139,face_forehead1,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na
TTGTGTTTCCCGAAAG-1,1,51,59,939,1411,80,5.669738,174,12.331679,TTGTGTTTCCCGAAAG-1,...,1.983099,CL:0000057,face_forehead1,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na


In [150]:
#### **tissue_type**

In [151]:
adata.obs['tissue_type'] = ['tissue'] * len(adata.obs)

In [152]:
adata.obs['tissue_type'] = adata.obs['tissue_type'].astype('category')

#### **tissue_ontology_term_id**

In [153]:
adata.obs['tissue_ontology_term_id'] = ['UBERON:0016475'] * len(adata.obs)

In [154]:
adata.obs['tissue_ontology_term_id'] = adata.obs['tissue_ontology_term_id'].astype('category')

In [155]:
list(adata.obs['tissue_ontology_term_id'].unique())

['UBERON:0016475']

In [156]:
# view obs

In [157]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data,organism_ontology_term_id,self_reported_ethnicity_ontology_term_id,sex_ontology_term_id,suspension_type,tissue_type,tissue_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAGCTTTCAGAAG-1,1,43,9,1848,3652,94,2.573932,552,15.115005,AAACAGCTTTCAGAAG-1,...,face_forehead1,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0016475
AAACAGGGTCTATATT-1,1,47,13,1877,3577,128,3.578418,537,15.012580,AAACAGGGTCTATATT-1,...,face_forehead1,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0016475
AAACCGGGTAGGTACC-1,1,42,28,3439,9152,211,2.305507,1982,21.656469,AAACCGGGTAGGTACC-1,...,face_forehead1,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0016475
AAACCGTTCGTCCAGG-1,1,52,42,1403,2472,128,5.177994,310,12.540454,AAACCGTTCGTCCAGG-1,...,face_forehead1,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0016475
AAACCTCATGAAGTTG-1,1,37,19,7836,32964,366,1.110302,6706,20.343405,AAACCTCATGAAGTTG-1,...,face_forehead1,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0016475
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTGGTAGGAGGGAT-1,1,50,28,953,1536,58,3.776042,294,19.140625,TTGTGGTAGGAGGGAT-1,...,face_forehead1,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0016475
TTGTGTATGCCACCAA-1,1,56,60,589,797,67,8.406525,92,11.543287,TTGTGTATGCCACCAA-1,...,face_forehead1,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0016475
TTGTGTTTCCCGAAAG-1,1,51,59,939,1411,80,5.669738,174,12.331679,TTGTGTTTCCCGAAAG-1,...,face_forehead1,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0016475


In [158]:
adata.obs.columns

Index(['in_tissue', 'array_row', 'array_col', 'n_genes_by_counts',
       'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb',
       'pct_counts_rb', 'barcode', 'c2l_Th', 'c2l_NK', 'c2l_APOD+ fibroblasts',
       'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2', 'c2l_DC1',
       'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes',
       'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc',
       'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes',
       'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts',
       'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC',
       'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes',
       'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1',
       'assay_ontology_term_id', 'max_c2l_column', 'max_c2l_column_value',
       'cell_type_ontology_term_id', 'donor_id',
       'development_stage_ontology_term_id', 'disease_ontology_term_id',
       'is_primary_data', 

In [159]:
del adata.obs['barcode']
del adata.obs['max_c2l_column']
del adata.obs['max_c2l_column_value']
del adata.obs['array_row']
del adata.obs['array_col']

#### **obsm (Embeddings)**

In [160]:
adata.obsm

AxisArrays with keys: X_spatial

In [161]:
adata.obsm.keys()

KeysView(AxisArrays with keys: X_spatial)

#### **uns (Dataset Metadata)**

In [162]:
adata.uns

OverloadedDict, wrapping:
	{'leiden': {'params': {'n_iterations': -1, 'random_state': 0, 'resolution': 1}}, 'log1p': {}, 'neighbors': {'connectivities_key': 'connectivities', 'distances_key': 'distances', 'params': {'method': 'umap', 'metric': 'euclidean', 'n_neighbors': 15, 'random_state': 0}}, 'pca': {'params': {'use_highly_variable': False, 'zero_center': True}, 'variance': array([715.15656 , 144.42233 , 104.09539 ,  75.74924 ,  61.33122 ,
        51.52883 ,  40.80014 ,  38.76496 ,  36.75816 ,  33.49591 ,
        30.704958,  29.978838,  29.41183 ,  29.306297,  28.081905,
        27.923721,  27.784176,  27.501026,  27.43014 ,  27.081274,
        27.0269  ,  26.855711,  26.71571 ,  26.522043,  26.354939,
        26.269175,  26.116476,  26.079197,  26.007427,  25.980713,
        25.85709 ,  25.80276 ,  25.744898,  25.639833,  25.593868,
        25.495707,  25.458622,  25.408   ,  25.3718  ,  25.153973,
        25.116611,  25.099573,  25.031763,  24.90443 ,  24.891138,
        24.825598

In [163]:
adata.uns['image_caption'] = 'Shown here is an image of 10 μm thick slice of the skin from the forehead stained with H&E'

In [164]:
adata.uns['title'] = 'Visium spatial - face_forehead_A_a'

In [165]:
adata.uns['default_embedding'] = 'X_spatial'

In [166]:
adata.uns.keys()

dict_keys(['leiden', 'log1p', 'neighbors', 'pca', 'spatial', 'umap', 'image_caption', 'title', 'default_embedding'])

### **Final checks and adjustments**

In [167]:
adata

AnnData object with n_obs × n_vars = 1163 × 36503
    obs: 'in_tissue', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb', 'pct_counts_rb', 'c2l_Th', 'c2l_NK', 'c2l_APOD+ fibroblasts', 'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2', 'c2l_DC1', 'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes', 'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc', 'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes', 'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts', 'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC', 'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes', 'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1', 'assay_ontology_term_id', 'cell_type_ontology_term_id', 'donor_id', 'development_stage_ontology_term_id', 'disease_ontology_term_id', 'is_primary_data', 'organism_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'sex_ontology_term_id', 'suspension_type', 'tissue_t

In [168]:
adata.obs.dtypes

in_tissue                                      int64
n_genes_by_counts                              int32
total_counts                                   int32
total_counts_mt                                int32
pct_counts_mt                                float32
total_counts_rb                                int32
pct_counts_rb                                float32
c2l_Th                                       float64
c2l_NK                                       float64
c2l_APOD+ fibroblasts                        float64
c2l_CD8+ T RM                                float64
c2l_T reg                                    float64
c2l_Macro1_2                                 float64
c2l_DC1                                      float64
c2l_SFRP2+ fibroblasts                       float64
c2l_TAGLN+ pericytes                         float64
c2l_POSTN+ fibroblasts                       float64
c2l_RGS5+ pericytes                          float64
c2l_VEC                                      f

In [169]:
dty = pd.DataFrame(adata.var.dtypes, columns = ['dtype'])
for c in dty[dty['dtype'] == 'float64'].index.values:
    adata.var[c] = adata.var[c].astype('float32')
    print(f"changed {c} from float64 to float32")
for c in dty[dty['dtype'] == 'int64'].index.values:
    adata.var[c] = adata.var[c].astype('int32') 
    print(f"changed {c} from int64 to int32")

In [170]:
dty = pd.DataFrame(adata.obs.dtypes, columns = ['dtype'])
for c in dty[dty['dtype'] == 'float64'].index.values:
    adata.obs[c] = adata.obs[c].astype('float32')
    print(f"changed {c} from float64 to float32")
for c in dty[dty['dtype'] == 'int64'].index.values:
    adata.obs[c] = adata.obs[c].astype('int32') 
    print(f"changed {c} from int64 to int32")
for c in dty[dty['dtype'] == 'object'].index.values:
    adata.obs[c] = adata.obs[c].astype('category') 
    print(f"changed {c} from object to category")

changed c2l_Th from float64 to float32
changed c2l_NK from float64 to float32
changed c2l_APOD+ fibroblasts from float64 to float32
changed c2l_CD8+ T RM from float64 to float32
changed c2l_T reg from float64 to float32
changed c2l_Macro1_2 from float64 to float32
changed c2l_DC1 from float64 to float32
changed c2l_SFRP2+ fibroblasts from float64 to float32
changed c2l_TAGLN+ pericytes from float64 to float32
changed c2l_POSTN+ fibroblasts from float64 to float32
changed c2l_RGS5+ pericytes from float64 to float32
changed c2l_VEC from float64 to float32
changed c2l_Tc from float64 to float32
changed c2l_ILC_NK from float64 to float32
changed c2l_BC from float64 to float32
changed c2l_Monocytes from float64 to float32
changed c2l_MastC from float64 to float32
changed c2l_Melanocytes from float64 to float32
changed c2l_DC2 from float64 to float32
changed c2l_LEC from float64 to float32
changed c2l_PlasmaC from float64 to float32
changed c2l_PTGDS+ fibroblasts from float64 to float32
chan

In [171]:
adata.obs

,in_tissue,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,c2l_Th,c2l_NK,c2l_APOD+ fibroblasts,...,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data,organism_ontology_term_id,self_reported_ethnicity_ontology_term_id,sex_ontology_term_id,suspension_type,tissue_type,tissue_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAGCTTTCAGAAG-1,1,1848,3652,94,2.573932,552,15.115005,0.037670,0.138531,0.637931,...,face_forehead1,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0016475
AAACAGGGTCTATATT-1,1,1877,3577,128,3.578418,537,15.012580,0.027854,0.182035,3.409482,...,face_forehead1,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0016475
AAACCGGGTAGGTACC-1,1,3439,9152,211,2.305507,1982,21.656469,0.137600,0.108754,3.759805,...,face_forehead1,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0016475
AAACCGTTCGTCCAGG-1,1,1403,2472,128,5.177994,310,12.540454,0.011250,0.009905,0.112893,...,face_forehead1,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0016475
AAACCTCATGAAGTTG-1,1,7836,32964,366,1.110302,6706,20.343405,0.895941,2.603344,0.208629,...,face_forehead1,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0016475
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTGGTAGGAGGGAT-1,1,953,1536,58,3.776042,294,19.140625,0.008805,0.012614,1.224670,...,face_forehead1,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0016475
TTGTGTATGCCACCAA-1,1,589,797,67,8.406525,92,11.543287,0.003327,0.012476,0.872187,...,face_forehead1,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0016475
TTGTGTTTCCCGAAAG-1,1,939,1411,80,5.669738,174,12.331679,0.006361,0.010583,1.022702,...,face_forehead1,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0016475


In [172]:
adata.obs.columns

Index(['in_tissue', 'n_genes_by_counts', 'total_counts', 'total_counts_mt',
       'pct_counts_mt', 'total_counts_rb', 'pct_counts_rb', 'c2l_Th', 'c2l_NK',
       'c2l_APOD+ fibroblasts', 'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2',
       'c2l_DC1', 'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes',
       'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc',
       'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes',
       'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts',
       'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC',
       'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes',
       'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1',
       'assay_ontology_term_id', 'cell_type_ontology_term_id', 'donor_id',
       'development_stage_ontology_term_id', 'disease_ontology_term_id',
       'is_primary_data', 'organism_ontology_term_id',
       'self_reported_ethnicity_ontology_term_id', 'sex_ontology

In [173]:
adata.var

,feature_is_filtered
ENSG00000243485,True
ENSG00000237613,True
ENSG00000186092,True
ENSG00000238009,False
ENSG00000239945,True
...,...
ENSG00000277836,True
ENSG00000278633,True
ENSG00000276017,True
ENSG00000278817,False


In [174]:
adata.obs

,in_tissue,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,c2l_Th,c2l_NK,c2l_APOD+ fibroblasts,...,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data,organism_ontology_term_id,self_reported_ethnicity_ontology_term_id,sex_ontology_term_id,suspension_type,tissue_type,tissue_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAGCTTTCAGAAG-1,1,1848,3652,94,2.573932,552,15.115005,0.037670,0.138531,0.637931,...,face_forehead1,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0016475
AAACAGGGTCTATATT-1,1,1877,3577,128,3.578418,537,15.012580,0.027854,0.182035,3.409482,...,face_forehead1,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0016475
AAACCGGGTAGGTACC-1,1,3439,9152,211,2.305507,1982,21.656469,0.137600,0.108754,3.759805,...,face_forehead1,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0016475
AAACCGTTCGTCCAGG-1,1,1403,2472,128,5.177994,310,12.540454,0.011250,0.009905,0.112893,...,face_forehead1,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0016475
AAACCTCATGAAGTTG-1,1,7836,32964,366,1.110302,6706,20.343405,0.895941,2.603344,0.208629,...,face_forehead1,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0016475
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTGGTAGGAGGGAT-1,1,953,1536,58,3.776042,294,19.140625,0.008805,0.012614,1.224670,...,face_forehead1,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0016475
TTGTGTATGCCACCAA-1,1,589,797,67,8.406525,92,11.543287,0.003327,0.012476,0.872187,...,face_forehead1,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0016475
TTGTGTTTCCCGAAAG-1,1,939,1411,80,5.669738,174,12.331679,0.006361,0.010583,1.022702,...,face_forehead1,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0016475


In [175]:
adata.obs.columns

Index(['in_tissue', 'n_genes_by_counts', 'total_counts', 'total_counts_mt',
       'pct_counts_mt', 'total_counts_rb', 'pct_counts_rb', 'c2l_Th', 'c2l_NK',
       'c2l_APOD+ fibroblasts', 'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2',
       'c2l_DC1', 'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes',
       'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc',
       'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes',
       'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts',
       'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC',
       'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes',
       'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1',
       'assay_ontology_term_id', 'cell_type_ontology_term_id', 'donor_id',
       'development_stage_ontology_term_id', 'disease_ontology_term_id',
       'is_primary_data', 'organism_ontology_term_id',
       'self_reported_ethnicity_ontology_term_id', 'sex_ontology

In [176]:
#check the format of expression matrix

In [177]:
adata.X

<1163x36503 sparse matrix of type '<class 'numpy.float32'>'
	with 19761696 stored elements in Compressed Sparse Row format>

In [178]:
araw.X

<1163x36503 sparse matrix of type '<class 'numpy.float32'>'
	with 2356147 stored elements in Compressed Sparse Row format>

In [179]:
#Copy raw counts to adata.raw

In [180]:
adata.raw = araw

In [ ]:
del adata.uns['log1p']

In [181]:
#write the curated object to final_objects folder

In [182]:
adata.write('/lustre/scratch127/cellgen/cellgeni/cxgportal_sets/spatial-skin/Final_objects/face_forehead1a.h5ad', compression = 'gzip')